In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully!")

In [ ]:
# Load the trained model
try:
    model_package = joblib.load('aftershock_prediction_model.pkl')
    
    # Extract components
    rf_model = model_package['model']
    feature_columns = model_package['feature_columns']
    magType_columns = model_package['magType_columns']
    performance = model_package['performance']
    
    print("   Model loaded successfully!")
    print(f"   Model Performance:")
    print(f"   • Accuracy: {performance['accuracy']:.3f}")
    print(f"   • F1-Score: {performance['f1_score']:.3f}")
    print(f"   • ROC-AUC: {performance['roc_auc']:.3f}")
    print(f"   Training Date: {model_package['training_date'][:10]}")
    
except FileNotFoundError:
    print("   Model file not found!")
    print("   Please run 'python train_model.py' first to train the model.")
except Exception as e:
    print(f"   Error loading model: {e}")

In [ ]:
def predict_aftershock_triggering(magnitude, depth, latitude, longitude, magType_str='mw',
                                 hour=12, day_of_week=1, month=6, year=2023,
                                 recent_activity=0, recent_avg_mag=0, recent_max_mag=0):
    """
    Predict whether an earthquake will trigger aftershocks
    
    Required Parameters:
    - magnitude: Earthquake magnitude (e.g., 6.5)
    - depth: Depth in kilometers (e.g., 15.0)
    - latitude: Latitude in decimal degrees (e.g., 34.052)
    - longitude: Longitude in decimal degrees (e.g., -118.243)
    - magType_str: Magnitude type ('mw', 'ml', 'mb', 'md', 'ms')
    
    Optional Parameters (use defaults if unknown):
    - hour: Hour of day (0-23)
    - day_of_week: Day of week (0=Monday, 6=Sunday)
    - month: Month (1-12)
    - year: Year
    - recent_activity: Number of recent earthquakes in area
    - recent_avg_mag: Average magnitude of recent earthquakes
    - recent_max_mag: Maximum magnitude of recent earthquakes
    
    Returns:
    - Dictionary with prediction and probability
    """
    
    if 'rf_model' not in globals():
        return {"error": "Model not loaded. Please run the model loading cell first."}
    
    try:
        # Create feature vector
        features = pd.DataFrame({
            'mag': [magnitude],
            'depth': [depth],
            'latitude': [latitude],
            'longitude': [longitude],
            'mag_squared': [magnitude ** 2],
            'depth_log': [np.log1p(depth)],
            'mag_depth_ratio': [magnitude / (depth + 1)],
            'hour': [hour],
            'day_of_week': [day_of_week],
            'month': [month],
            'year': [year],
            'recent_activity_30d': [recent_activity],
            'recent_avg_mag_30d': [recent_avg_mag],
            'recent_max_mag_30d': [recent_max_mag],
            'lat_abs': [abs(latitude)],
            'distance_from_equator': [abs(latitude)]
        })
        
        # Handle magType encoding
        for col in magType_columns:
            mag_type = col.replace('magType_', '')
            features[col] = [1 if magType_str.lower() == mag_type.lower() else 0]
        
        # Ensure all required features are present
        for col in feature_columns:
            if col not in features.columns:
                features[col] = 0
        
        # Select features in correct order
        X_pred = features[feature_columns]
        
        # Make prediction
        prediction = rf_model.predict(X_pred)[0]
        probability = rf_model.predict_proba(X_pred)[0, 1]
        
        # Determine risk level
        if probability >= 0.7:
            risk_level = "   HIGH RISK"
            advice = "Expect significant aftershock activity"
        elif probability >= 0.4:
            risk_level = "   MODERATE RISK"
            advice = "Possible aftershock activity, monitor closely"
        else:
            risk_level = "   LOW RISK"
            advice = "Minimal aftershock activity expected"
        
        return {
            'will_trigger_aftershocks': bool(prediction),
            'probability': float(probability),
            'probability_percent': f"{probability:.1%}",
            'risk_level': risk_level,
            'advice': advice,
            'confidence': 'High' if probability > 0.7 or probability < 0.3 else 'Moderate'
        }
        
    except Exception as e:
        return {"error": f"Prediction failed: {str(e)}"}

print("🔮 Prediction function ready!")

In [ ]:
# Example 1: Large shallow earthquake (likely to trigger aftershocks)
print("   EXAMPLE 1: Large Shallow Earthquake")
print("=" * 40)

result1 = predict_aftershock_triggering(
    magnitude=5.2,
    depth=20,
    latitude=34.954,
    longitude=-5.635,
    magType_str='mw'
)

print(f"   Input: Magnitude 7.2 (Mw), Depth 8.5 km")
print(f"   Location: 34.052°N, 118.243°W (Los Angeles area)")
print(f"\n   Prediction Results:")
print(f"   • Will trigger aftershocks: {result1['will_trigger_aftershocks']}")
print(f"   • Probability: {result1['probability_percent']}")
print(f"   • Risk Level: {result1['risk_level']}")
print(f"   • Advice: {result1['advice']}")
print(f"   • Confidence: {result1['confidence']}")

In [ ]:
# Example 2: Moderate deep earthquake (less likely to trigger aftershocks)
print("\n   EXAMPLE 2: Moderate Deep Earthquake")
print("=" * 40)

result2 = predict_aftershock_triggering(
    magnitude=5.8,
    depth=120.0,
    latitude=40.123,
    longitude=-75.456,
    magType_str='mb'
)

print(f"   Input: Magnitude 5.8 (mb), Depth 120.0 km")
print(f"   Location: 40.123°N, 75.456°W")
print(f"\n   Prediction Results:")
print(f"   • Will trigger aftershocks: {result2['will_trigger_aftershocks']}")
print(f"   • Probability: {result2['probability_percent']}")
print(f"   • Risk Level: {result2['risk_level']}")
print(f"   • Advice: {result2['advice']}")
print(f"   • Confidence: {result2['confidence']}")

In [ ]:
#    CUSTOMIZE YOUR EARTHQUAKE SCENARIO HERE:
# ============================================

# Required parameters (MODIFY THESE):
my_magnitude = 4.3      # Change this value
my_depth = 7.8         # Change this value
my_latitude = 37.867      # Change this value
my_longitude = -122.290   # Change this value
my_magType = 'mw'       # Options: 'mw', 'ml', 'mb', 'md', 'ms'

# Optional parameters (leave as default or modify):
my_hour = 14            # Hour of day (0-23)
my_day_of_week = 2      # Day of week (0=Monday, 6=Sunday)
my_month = 8            # Month (1-12)
my_year = 2024          # Year

# Make prediction
my_result = predict_aftershock_triggering(
    magnitude=my_magnitude,
    depth=my_depth,
    latitude=my_latitude,
    longitude=my_longitude,
    magType_str=my_magType,
    hour=my_hour,
    day_of_week=my_day_of_week,
    month=my_month,
    year=my_year
)

# Display results
print("   YOUR CUSTOM EARTHQUAKE PREDICTION")
print("=" * 45)
print(f"   Input Parameters:")
print(f"   • Magnitude: {my_magnitude} ({my_magType.upper()})")
print(f"   • Depth: {my_depth} km")
print(f"   • Location: {my_latitude}°, {my_longitude}°")
print(f"   • Time: {my_hour}:00, Day {my_day_of_week}, Month {my_month}, Year {my_year}")

print(f"\n   Prediction Results:")
if 'error' in my_result:
    print(f"   Error: {my_result['error']}")
else:
    print(f"   • Will trigger aftershocks: {'YES' if my_result['will_trigger_aftershocks'] else 'NO'}")
    print(f"   • Probability: {my_result['probability_percent']}")
    print(f"   • Risk Level: {my_result['risk_level']}")
    print(f"   • Advice: {my_result['advice']}")
    print(f"   • Confidence: {my_result['confidence']}")

In [ ]:
# QUICK PREDICTION TEMPLATE - Copy and modify this block
def quick_predict(mag, depth, lat, lon, mag_type='mw', description=""):
    """Quick prediction with minimal parameters"""
    result = predict_aftershock_triggering(mag, depth, lat, lon, mag_type)
    
    print(f"   {description}")
    print(f"   M{mag} at {depth}km depth ({lat}°, {lon}°)")
    print(f"   Result: {'WILL trigger aftershocks' if result['will_trigger_aftershocks'] else 'unlikely to trigger aftershocks'}")
    print(f"   Probability: {result['probability_percent']} | {result['risk_level']}")
    print("─" * 50)

# Example usage - uncomment and modify these lines:

# quick_predict(7.0, 15.0, 34.0, -118.0, 'mw', "Major LA earthquake")
# quick_predict(5.5, 80.0, 40.0, -120.0, 'ml', "Moderate deep earthquake")
# quick_predict(6.2, 5.0, 38.0, -122.0, 'mw', "Shallow Bay Area earthquake")

print("   Quick prediction template ready!")
print("   Uncomment and modify the example lines above to test different scenarios.")

In [ ]:
# Display model information
if 'model_package' in globals():
    print("   MODEL INFORMATION")
    print("=" * 30)
    
    data_info = model_package['data_info']
    performance = model_package['performance']
    
    print(f"   Training Data:")
    print(f"   • Total earthquakes: {data_info['total_samples']:,}")
    print(f"   • Training samples: {data_info['training_samples']:,}")
    print(f"   • Test samples: {data_info['test_samples']:,}")
    print(f"   • Date range: {data_info['date_range']['start'][:10]} to {data_info['date_range']['end'][:10]}")
    
    print(f"\n   Performance Metrics:")
    print(f"   • Accuracy: {performance['accuracy']:.3f} ({performance['accuracy']:.1%})")
    print(f"   • F1-Score: {performance['f1_score']:.3f}")
    print(f"   • ROC-AUC: {performance['roc_auc']:.3f}")
    print(f"   • Out-of-bag Score: {performance['oob_score']:.3f}")
    
    print(f"\n   Model Features: {len(feature_columns)} total")
    print(f"   • Basic: magnitude, depth, location")
    print(f"   • Derived: magnitude², log(depth), ratios")
    print(f"   • Temporal: hour, day, month, year")
    print(f"   • Regional: recent seismic activity")
    print(f"   • Geographic: latitude-based features")
    print(f"   • Categorical: magnitude types")
    
    print(f"\n   Model Training Date: {model_package['training_date'][:19]}")
    
else:
    print("   Model not loaded. Run the model loading cell first.")